In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cluster import KMeans
from xgboost import XGBClassifier, XGBRegressor

import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, UpSampling2D, Input, Rescaling, BatchNormalization
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Anemia prediction (Classification & Regression)

In [ ]:
mode = "classification"
#mode = "regression"
binary = True
threshold = [7.0, 10.0, 12.5]
threshold_name = ["severely anemic", "moderately anemic", "mildly anemic", "non-anemic"]

## Load labels

In [ ]:
label_path = os.path.join("D:", "OneDrive_1_5-26-2022", "PredictingAnemia_DATA_2022-06-05_0643.csv")
label = pd.read_csv(label_path)

In [ ]:
label["hgb"] = pd.to_numeric(label["hgb"], errors="coerce")
drop_index = np.where(pd.isnull(label["hgb"]))
print("drop (contains string or null): ", drop_index[0])
label = label.drop(drop_index[0])
print("mean:", label["hgb"].mean(), "std:", label["hgb"].std())
print("anemia mean: ", label["hgb"][label["hgb"] < 12.5].mean())
print("non-anemia mean: ", label["hgb"][label["hgb"] >= 12.5].mean())

In [ ]:
def multi_class_label(label_i, threshold):
    label = -1
    for i, threshold_i in enumerate(threshold):
        if label_i < threshold_i:
            label = i
            break
    if label == -1:
        label = len(threshold)
    
    #print(label, label_i)
    
    return label

if mode == "classification":
    if binary:
        y = (label["hgb"] < 12.5).astype(int)
    else:
        y = np.array([multi_class_label(label_i, threshold) for label_i in label["hgb"]], dtype=np.uint8)
        y = pd.Series(data=y, index=label["hgb"].index)
elif mode == "regression":
    y = label["hgb"]
print(y.index)

In [ ]:
y_available = [] 

for folder in os.listdir("./detected eyes images"):
    if int(folder)-1 in y.index:
        y_available.append(int(folder))
        
print("not available id: ")
not_available_id = []
for i in range(1, 693):
    if i not in y_available:
        not_available_id.append(i)
print(not_available_id)
print("num: ", len(not_available_id))

## Load images

In [ ]:
x_img = []
y_img = []
y_id = []

for id in y_available:
    for image in os.listdir(os.path.join("./detected eyes images", str(id))):
        #print(id, image)
        img = cv2.imread(os.path.join("./detected eyes images", str(id), image))
        #print(img.shape)
        x_img.append(tf.image.resize(img, (224, 224)))
        y_img.append(y[id-1])
        y_id.append(id)
        
x_img = np.array(x_img, dtype=np.uint8)
y_img = np.array(y_img)
print(x_img.shape, y_img.shape)

## Data preprocessing

### Autoenconder

In [ ]:
def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 224, 224, 3))
    return array.astype(np.float32)

def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)

def display(array1, array2):
    """
    Displays ten random images from each one of the supplied arrays.
    """

    n = 10

    indices = np.random.randint(len(array1), size=n)
    
    images1 = np.zeros_like(array1[indices, :])
    images2 = np.zeros_like(array2[indices, :])
    images1[:, :, :, 0] = array1[indices, :, :, 2]
    images1[:, :, :, 1] = array1[indices, :, :, 1]
    images1[:, :, :, 2] = array1[indices, :, :, 0]
    images2[:, :, :, 0] = array2[indices, :, :, 2]
    images2[:, :, :, 1] = array2[indices, :, :, 1]
    images2[:, :, :, 2] = array2[indices, :, :, 0]

    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

x_img_preprocess = preprocess(x_img)
x_img_noise = noise(x_img_preprocess)

In [ ]:
class Vgg16(tf.keras.Model):
    def __init__(self, pretrained = True):
        super(Vgg16, self).__init__()
        self.vggnet = tf.keras.applications.VGG16(include_top=False, weights=None)
        #features = list(self.vggnet.features)
        #self.layers = tf.keras.Sequential(features).eval() 
        
    def call(self, x):
        results = []
        for ii,model in enumerate(self.vggnet.layers):
            x = model(x)
            if ii in [2,5,9,13,17]:
                results.append(x) #(64,256,256),(128,128,128),(256,64,64),(512,32,32),(512,16,16)
        return results

vgg_model = Vgg16()
vgg_model.build(input_shape=(None, 224, 224, 3))
vgg_model.summary()

class DeConv2d(tf.keras.layers.Layer):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, dilation):
        super().__init__()
        self.up = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='nearest')
        self.conv = tf.keras.layers.Conv2D(filters=out_channel, kernel_size=kernel_size, strides=stride, padding=padding, dilation_rate=dilation)
    
    def call(self, x):
        output = self.up(x)
        output = self.conv(output)
        return output

class UNet(tf.keras.Model):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        #####################################
        self.relu = tf.keras.layers.ReLU()
        self.deconv1 = DeConv2d(512, 512, kernel_size=3, stride=1, padding="same", dilation=1)
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.deconv2 = DeConv2d(1024, 256, kernel_size=3, stride=1, padding="same", dilation=1)
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.deconv3 = DeConv2d(512, 128, kernel_size=3, stride=1, padding="same", dilation=1)
        self.bn3 = tf.keras.layers.BatchNormalization()
        
        self.deconv4 = DeConv2d(256, 64, kernel_size=3, stride=1, padding="same", dilation=1)
        self.bn4 = tf.keras.layers.BatchNormalization()
        
        self.classifier = tf.keras.layers.Conv2D(n_class, kernel_size=1, activation="sigmoid")
        #####################################
    
    def call(self, x):
        #####################################
        pre_output = self.pretrained_net(x)
        output = self.bn1(self.relu(self.deconv1(pre_output[4]))) #(512,32,32)
        output = self.bn2(self.relu(self.deconv2(tf.concat([output, pre_output[3]], axis=-1)))) #(256,64,64)
        output = self.bn3(self.relu(self.deconv3(tf.concat([output, pre_output[2]], axis=-1)))) #(128,128,128)
        output = self.bn4(self.relu(self.deconv4(tf.concat([output, pre_output[1]], axis=-1)))) #(64,256,256)
        output = self.classifier(tf.concat([output, pre_output[0]], axis=-1))
        return output
        #####################################
        
seg_model = UNet(pretrained_net=vgg_model, n_class=3)
seg_model.compile(optimizer='adam', loss='binary_crossentropy')
seg_model.build(input_shape=(None, 224, 224, 3))
seg_model.summary()

seg_model.fit(
    x=x_img_preprocess,
    y=x_img_preprocess,
    epochs=10,
    batch_size=16,
    shuffle=True,
)

x_img_denoise = seg_model.predict(x_img_preprocess, batch_size=16)
display(x_img_preprocess, x_img_noise)
display(x_img_preprocess, x_img_denoise)

In [ ]:
inputs = Input(shape=(224, 224, 3))
# Encoder
x = Conv2D(256, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
# Decoder
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(inputs, x)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

autoencoder.fit(
    x=x_img_preprocess,
    y=x_img_preprocess,
    epochs=10,
    batch_size=16,
    shuffle=True,
)

x_img_denoise = autoencoder.predict(x_img_preprocess, batch_size=16)
display(x_img_preprocess, x_img_noise)
display(x_img_preprocess, x_img_denoise)

In [ ]:
x_img = np.array(x_img_denoise * 255.0, copy=True, dtype=np.uint8)

### Changing the contrast and brightness

In [ ]:
lookUpTable = np.empty((1,256), np.uint8)
gamma = 1.3
for i in range(256):
    lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)

def adjust_brightness(img, lookUpTable, alpha=1.3, beta=40):
    new_image = np.zeros(img.shape, img.dtype)
    
    #for y in range(img.shape[0]):
    #    for x in range(img.shape[1]):
    #        for c in range(img.shape[2]):
    #            new_image[y,x,c] = np.clip(alpha*img[y,x,c] + beta, 0, 255)

    new_image = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
                
    res = cv2.LUT(new_image, lookUpTable)
                
    return res

# x_brightness = np.array([adjust_brightness(xi, lookUpTable) for xi in x_img], dtype=np.uint8)
# print(x_brightness.shape)

# x_preprocessed = np.array(x_brightness, copy=True)

In [ ]:
# frame = x_img[10]
# result = x_brightness[10]

# cv2.imshow('frame', frame)
# cv2.imshow('result', result)

# cv2.waitKey(0)

# cv2.destroyAllWindows()

### Clustering filter

In [ ]:
def clustering_filter(img, n_clusters=5):
    original_shape = img.shape
    img = img.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(img)

    labels=kmeans.labels_
    #print(labels)
    labels=list(labels)

    centroid=kmeans.cluster_centers_
    #print(centroid)

    percent=[]
    for i in range(len(centroid)):
      j=labels.count(i)
      j=j/(len(labels))
      percent.append(j)
    #print(percent)

    # bgr to rgb
    #plt.pie(percent,colors=np.array(centroid[:, [2, 1, 0]]/255),labels=np.arange(len(centroid)))
    #plt.show()

    sorted_percent = sorted(percent)
    remove_index = [percent_i in [sorted_percent[0], sorted_percent[1]] for percent_i in percent]
    #print(remove_index)

    result = np.array(img, copy=True)
    for i, remove in enumerate(remove_index):
        if remove:
            result[labels==np.array(i)] = centroid[i]
    result = result.reshape(original_shape)
    
    return result

# x_cluster = np.array([clustering_filter(xi) for xi in x_img], dtype=np.uint8)
# print(x_cluster.shape)

# x_preprocessed = np.array(x_cluster, copy=True)

In [ ]:
# frame = x_img[321]
# result = x_cluster[321]

# cv2.imshow('frame', frame)
# cv2.imshow('result', result)

# cv2.waitKey(0)

# cv2.destroyAllWindows()

### HSV filter

In [ ]:
dummy_sum = []

def hsv_filter(img, init_value=100, end_value=0, average_value=20000, adaptive=False):
    mask_value = 0
    sv_value = init_value
    
    if adaptive:
        while mask_value <= average_value and sv_value >= end_value:
            # Threshold of blue in HSV space
            lower_red = np.array([0,sv_value,sv_value])
            upper_red = np.array([10,255,255])
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            # preparing the mask to overlay
            mask = cv2.inRange(hsv, lower_red, upper_red)
            mask_value = np.sum(mask/255)
            sv_value -= 1
    else:
        lower_red = np.array([0,sv_value,sv_value])
        upper_red = np.array([10,255,255])
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        # preparing the mask to overlay
        mask = cv2.inRange(hsv, lower_red, upper_red)
        mask_value = np.sum(mask/255)
        
    dummy_sum.append(mask_value)


    # The black region in the mask has the value of 0,
    # so when multiplied with original image removes all non-blue regions
    result = cv2.bitwise_and(img, img, mask = mask)
    
    return result

x_hsv = np.array([hsv_filter(xi) for xi in x_img], dtype=np.uint8)
print(x_hsv.shape)

print(np.mean(np.array(dummy_sum), axis=0))

x_preprocessed = np.array(x_hsv, copy=True)

In [ ]:
# frame = x_img[60]
# result = x_hsv[60]

# cv2.imshow('frame', frame)
# cv2.imshow('result', result)

# cv2.waitKey(0)

# cv2.destroyAllWindows()

### Histrogram

In [ ]:
# Blue, Green, Red and A (Transparency)
def red_histogram(img):
    return np.histogram(img[:, :, 2].flatten(), range(257))[0]

x_hist = np.array([red_histogram(xi) for xi in x_preprocessed])
print(x_hist.shape)

x_final = x_hist
y_final = y_img

In [ ]:
plt.bar(np.arange(256)[1:] - 0.5, x_hist[0][1:], width=1, edgecolor='none')
plt.xlim([-0.5, 255.5])
plt.show()

## Train test split

In [ ]:
# split by id
x_train, y_train = x_final[[i > 80 for i in y_id]], y_final[[i > 80 for i in y_id]]
x_test, y_test = x_final[[i <= 80 for i in y_id]], y_final[[i <= 80 for i in y_id]]

#x_train, y_train = x_final[[i < 620 for i in y_id]], y_final[[i < 620 for i in y_id]]
#x_test, y_test = x_final[[i >= 620 for i in y_id]], y_final[[i >= 620 for i in y_id]]

if mode == "classification":
    if binary:
        print("train: (0)", np.sum(y_train==0), "(1)", np.sum(y_train==1))
        print("test: (0)", np.sum(y_test==0), "(1)", np.sum(y_test==1))
    else:
        for i in range(len(threshold)+1):
            print(i)
            print("train:", np.sum(y_train==i), " test:", np.sum(y_test==i))
elif mode == "regression":
    print(np.mean(y_train))
    print(np.mean(y_test))

In [ ]:
# normalization
# scaler = RobustScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

In [ ]:
if mode == "classification":
    if binary:
        plt.plot(np.arange(256)[1:] - 0.5, x_train[y_train==0].mean(axis=0)[1:], label='non-anemia')
        plt.plot(np.arange(256)[1:] - 0.5, x_train[y_train==1].mean(axis=0)[1:], label='anemia')
    else:
        for i in range(len(threshold)+1):
            plt.plot(np.arange(256)[1:] - 0.5, x_train[y_train==i].mean(axis=0)[1:], label=threshold_name[i])
elif mode == "regression":
    plt.plot(np.arange(256)[1:] - 0.5, x_train[y_train>=12.5].mean(axis=0)[1:], label='non-anemia')
    plt.plot(np.arange(256)[1:] - 0.5, x_train[y_train<12.5].mean(axis=0)[1:], label='anemia')
plt.legend()
plt.show()

In [ ]:
if mode == "classification":
    if binary:
        plt.plot(np.arange(256)[1:] - 0.5, x_test[y_test==0].mean(axis=0)[1:], label='non-anemia')
        plt.plot(np.arange(256)[1:] - 0.5, x_test[y_test==1].mean(axis=0)[1:], label='anemia')
    else:
        for i in range(len(threshold)+1):
            plt.plot(np.arange(256)[1:] - 0.5, x_test[y_test==i].mean(axis=0)[1:], label=threshold_name[i])
elif mode == "regression":
    plt.plot(np.arange(256)[1:] - 0.5, x_test[y_test>=12.5].mean(axis=0)[1:], label='non-anemia')
    plt.plot(np.arange(256)[1:] - 0.5, x_test[y_test<12.5].mean(axis=0)[1:], label='anemia')
plt.legend()
plt.show()

## Classifiers

In [ ]:
def print_results(y_true, y_hat, mode, binary):
    if mode == "classification":
        if binary:
            average = "binary"
            multi_class = "raise"
            display_labels = ["non-anemia", "anemia"]
        else:
            average = "macro"
            multi_class = "ovo"
            display_labels = threshold_name
            
        print("accuracy: ", accuracy_score(y_true, np.argmax(y_hat, axis=1)))
        print("precision: ", precision_score(y_true, np.argmax(y_hat, axis=1), average=average))
        print("recall: ", recall_score(y_true, np.argmax(y_hat, axis=1), average=average))
        if binary:
            print("roc auc: ", roc_auc_score(y_true, y_hat[:, 1], multi_class=multi_class))
        else:
            print("roc auc: ", roc_auc_score(y_true, y_hat, multi_class=multi_class))
        print("f1: ", f1_score(y_true, np.argmax(y_hat, axis=1), average=average))
        print("cohen kappa score: ", cohen_kappa_score(y_true, np.argmax(y_hat, axis=1)))
        y_hat = np.argmax(y_hat, axis=1)
        cm = confusion_matrix(y_true, y_hat)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
        disp.plot()
        plt.show()
    elif mode == "regression":
        print("mse: ", mean_squared_error(y_true, y_hat))
        print("mae: ", mean_absolute_error(y_true, y_hat))

In [ ]:
if mode == "classification":
    clf = LogisticRegression(random_state=0, max_iter=10000, solver='saga')
    clf.fit(x_train, y_train)
    print("Logistic regression")
    print_results(y_test, clf.predict_proba(x_test), mode, binary)

    clf = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=0)
    clf.fit(x_train, y_train)
    print("Random forest")
    print_results(y_test, clf.predict_proba(x_test), mode, binary)

    clf = SVC(random_state=0, probability=True)
    clf.fit(x_train, y_train)
    print("SVM")
    print_results(y_test, clf.predict_proba(x_test), mode, binary)

    clf = XGBClassifier(random_state=0)
    clf.fit(x_train, y_train)
    print("XGBoost")
    print_results(y_test, clf.predict_proba(x_test), mode, binary)
elif mode == "regression":
    clf = LinearRegression()
    clf.fit(x_train, y_train)
    print("Linear regression")
    print_results(y_test, clf.predict(x_test), mode, binary)
    
    clf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
    clf.fit(x_train, y_train)
    print("Random forest")
    print_results(y_test, clf.predict(x_test), mode, binary)
    
    clf = SVR()
    clf.fit(x_train, y_train)
    print("SVM")
    print_results(y_test, clf.predict(x_test), mode, binary)
    
    clf = XGBRegressor(n_estimators=10, random_state=0)
    clf.fit(x_train, y_train)
    print("XGBoost")
    print_results(y_test, clf.predict(x_test), mode, binary)

## Visualization

In [ ]:
if mode == "classification":
    x_img_train, y_img_train = x_img[[i > 80 for i in y_id]], y_img[[i > 80 for i in y_id]]
    x_img_test, y_img_test = x_img[[i <= 80 for i in y_id]], y_img[[i <= 80 for i in y_id]]

    predictions = np.argmax(clf.predict_proba(x_test), axis=1)
    correct_index = predictions == y_test
    for i, correct in enumerate(correct_index):
        if correct:
            correct_img = x_img_test[i]
            correct_img = correct_img.astype(np.uint8)
            cv2.putText(correct_img, str(y_test[i]), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.imshow("correct", correct_img)
            cv2.waitKey(500)
    cv2.destroyAllWindows()

In [ ]:
if mode == "classification":
    wrong_index = predictions != y_test
    for i, wrong in enumerate(wrong_index):
        if wrong:
            wrong_img = x_img_test[i]
            wrong_img = wrong_img.astype(np.uint8)
            cv2.putText(wrong_img, str(y_test[i]), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.imshow("wrong", wrong_img)
            cv2.waitKey(500)
    cv2.destroyAllWindows()